In [ ]:
%pip install -r ../requirements.txt

In [ ]:
import lightning as L

# set seed for reproducibility
L.seed_everything(seed=42, workers=True)

In [3]:
from Classifier import ChessClassifier

model = ChessClassifier(in_features=2*6*8*8, labels=3)

In [ ]:
import pandas as pd

df = pd.read_csv('data/lichess/data.csv')

# fix differing occurence frequencys of labels
print('Distribution of labels:')
print(df['TurnEvaluation'].value_counts(normalize=True))

min_count = df['TurnEvaluation'].value_counts().min()
equalized_subset = df.groupby('TurnEvaluation').apply(lambda x: x.sample(n=min_count)).reset_index(drop=True)

print("Equalized distribution of labels:")
print(equalized_subset['TurnEvaluation'].value_counts(normalize=True))

print(f'New length of dataset: {len(equalized_subset)}')
df = equalized_subset

In [5]:
import torch.utils.data as tdata
import pandas as pd

from Encoder import ColorBased
from Dataset import DataframeDataset


dataset = DataframeDataset(
    df['FEN'],
    df['PropableWinner'],
    feature_transform=ColorBased.fen_to_tensor,
    label_transform=ColorBased.label_to_tensor)

batchsize = 126
val_len = 10000
test_len = 10000
train_len = len(dataset) - val_len - test_len

train_data, val_data, test_data = tdata.random_split(dataset, (train_len, val_len, test_len))

train_loader = tdata.DataLoader(train_data, batch_size=batchsize, shuffle=True, num_workers=4, persistent_workers=True)
val_loader = tdata.DataLoader(val_data, batch_size=batchsize, shuffle=False, num_workers=4, persistent_workers=True)
test_loader = tdata.DataLoader(test_data, batch_size=batchsize, shuffle=False, num_workers=4, persistent_workers=True)

# delete df to clear up space
del df

In [6]:
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger

%load_ext tensorboard
tb_logger = TensorBoardLogger(save_dir='./logs/', name='TurnBasedClassifier')

In [ ]:
max_epochs=200
trainer = L.Trainer(
    limit_val_batches=0.2,
    max_epochs=max_epochs,
    logger=tb_logger
    )

trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

In [ ]:
trainer.test(model=model, dataloaders=test_loader)